In [4]:
import sys
sys.path.append('..\..\src') # this ensures that the src models are located to import them bellow
#sys.path.append('C:\\Users\\Marios\\Desktop\\Μάριος\\BotBattleOfSea\\src') deprecated absolute path

In [5]:
import cv2
import numpy as np
import pyautogui  # Import pyautogui for screen capture
from ultralytics import YOLO
from ultralytics.utils.plotting import colors, Annotator
from PIL import ImageGrab
from win32api import GetSystemMetrics
import keyboard #pip install keyboard
import time
from Paths import *

We will hardwrite all Tools function because we cant use from docs the function getText

In [30]:
def click(x1, x2, y1, y2):
    middle_x = (x1 + x2) / 2
    middle_y = (y1 + y2) / 2
    pyautogui.click(middle_x, middle_y)
    print("function: click") # for debugging only
    time.sleep(5) # for debugging only

def screenshot(x1, y1, x2, y2,i):
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    # Take a screenshot of the specified region
    screenshot = pyautogui.screenshot(region=(x1,y1,x2-x1,y2-y1))

    # Save the screenshot
    screenshot.save(f'OcrTexts\screenshot{i}.png')
    print("function: screenshot") # for debugging only
    time.sleep(5) # for debugging only

def screenshot_array(x1, y1, x2, y2,i):
    # Convert coordinates to integers
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    # Take a screenshot of the specified region
    screenshot = pyautogui.screenshot(region=(x1,y1,x2-x1,y2-y1))
    screenshot.save(f'OcrTexts\screenshot{i}.png')

    print("function: screenshot_array") # for debugging only
    time.sleep(5) # for debugging only
    return (np.array(screenshot))



#this absolute path works only for ipynb and not for the actual Tools.py file
def getText(screenshot_array_format,modelOCR=YOLO(absolute_path_object_model_1640px_windows)):
    results = modelOCR.predict(screenshot_array_format,imgsz=300, conf=0.2)
    boxes = results[0].boxes.xyxy.cpu()
    clss = results[0].boxes.cls.cpu().tolist()
    Dictionary = results[0].names
    mapping = {}
    for box, cls in zip(boxes, clss):
        mapping[box[0]] = Dictionary[cls][-1]
        #print(f"box={box[0]}, cls={cls}->{Dictionary[cls]}")
    
    sorted_dict = {k: v for k, v in sorted(mapping.items(), key=lambda item: item[0])}
    string=''
    for value in sorted_dict.values():
        string=string+value

    print("function: getText") # for debugging only
    time.sleep(5) # for debugging only
    return string

def mouse_in_safe_zone():
    pyautogui.FAILSAFE = False  # Disables the fail-safe feature
    pyautogui.moveTo(0, 0, duration=0)
    print("function: mouse_in_safe_zone") # for debugging only
    time.sleep(5) # for debugging only

import random

def getWidth_Height(x1, x2, y1, y2):
    # Ensure x1 < x2 and y1 < y2
    x_min, x_max = min(x1, x2), max(x1, x2)
    y_min, y_max = min(y1, y2), max(y1, y2)
    
    Width=x_max-x_min
    Height=y_max-y_min

    print("function: getWidth_Height") # for debugging only
    time.sleep(5) # for debugging only
    return Width, Height

def getMiddle_point(x1, x2, y1, y2):
    middle_x = (x1 + x2) / 2
    middle_y = (y1 + y2) / 2
    
    print("function: getMiddle_point") # for debugging only
    time.sleep(5) # for debugging only
    return(middle_x, middle_y)

def stableCam():
    pyautogui.typewrite('ws')
    print("function: stableCam") # for debugging only
    time.sleep(5) # for debugging only

def centerCamera(): # g is default letter for centering the cammera 
    pyautogui.typewrite('g')
    print("function: centerCamera") # for debugging only
    time.sleep(5) # for debugging only

We will make a loop only for ocr detection

In [34]:
model = YOLO(absolute_path_object_model_1640px_windows)
modelOCR=YOLO(absolute_path_ocr_model_400px_windows)
names = model.model.names

text=''
confirm=False
enable_to_click_glitter=True
click_next_glitter=True
navigation=False
height, width = GetSystemMetrics(0), GetSystemMetrics(1)
# Set the center point for the visioneye annotation
center_point = (-10, pyautogui.size()[1])  # Use the screen height as the y-coordinate

run=True
i=1
while run:

    img = ImageGrab.grab(bbox=(0,0, width, height))
    img_np = np.array(img)
    screen_bgr = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

    # Predict objects in the screenshot using the YOLO model
    results = model.predict(screen_bgr, conf=0.2)
    boxes = results[0].boxes.xyxy.cpu()
    clss = results[0].boxes.cls.cpu().tolist()
    Dictionary = results[0].names


    #prediction for Navigation
    resultsNavigation = model.predict(screen_bgr , imgsz=360, conf=0.2)
    boxesNavigation = results[0].boxes.xyxy.cpu()
    clssNavigation = results[0].boxes.cls.cpu().tolist()
    DictionaryNavigation = results[0].names



    #print(Dictionary.items())
    # Annotate the screenshot with bounding boxes and class labels
    annotator = Annotator(screen_bgr, line_width=2)
    for box, cls in zip(boxes, clss):
        #print(f"box={box}")
        #print(f"cls={cls}->{Dictionary[cls]}")
        print(f"**************detection********************box={box[0]}, cls={cls}->{Dictionary[cls]}")   
        time.sleep(5) # for debugging only

        if(cls==12.0): #12: ocrText
            #print(f'box[0]={box[0]},box[1]={box[1]},box[2]={box[2]},box[3]={box[3]}')
            Numbers=screenshot_array(box[0],box[1],box[2],box[3],i)
            text=getText(Numbers)
            i=i+1
            print(f'text={text} and object={Dictionary[cls]}')
        ##################################################################        OCR CAPTURE CODE       ###########################################################################
        if(cls==5.0 and text!=''): #5.0: TextFieldToBotQuestion
            click(box[0],box[2],box[1],box[3])   
            time.sleep(1) 
            pyautogui.typewrite(text)
            time.sleep(1)
            text=''
            print(f'text={text}')
            confirm=True
            #run = False   
        if(cls==4.0 and confirm): #4.0: ConfirmToBotQuestion
            confirm=False
            click(box[0],box[2],box[1],box[3])
        #############################################################################################################################################################################
        if(cls==9.0): #9.0: glitterClicked
            centerCamera()
            time.sleep(2) 
            break
        elif(cls==8.0): #8.0: glitter
            stableCam()
            click(box[0],box[2],box[1],box[3])
            mouse_in_safe_zone()
            


        
                
            

        annotator.box_label(box, label=names[int(cls)], color=colors(int(cls)))
        annotator.visioneye(box, center_point)

    if keyboard.is_pressed('esc'):
            print("Stop Loop!")
            run = False

# Close all OpenCV windows
cv2.destroyAllWindows()

KeyboardInterrupt: 